<a href="https://www.kaggle.com/code/uniquejie/notebookd1?scriptVersionId=158906883" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install mat73
!pip install github
!pip install pygithub

!python /kaggle/input/learning-code-v1/learning_code/main_2d.py
!python /kaggle/input/learning-code-v1/learning_code/upload.py

  Obtaining dependency information for mat73 from https://files.pythonhosted.org/packages/d4/c3/123c118e31cd58f5db333ae2750da13da0d87707adac2255357de9bb5f87/mat73-0.62-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.4 MB/s eta 0:00:00
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.2.0
    Uninstalling charset-normalizer-3.2.0:
      Successfully uninstalled charset-normalizer-3.2.0
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.8.5
    Uninstalling aiohttp-3.8.5:
      Successfully uninstalled aiohttp-3.8.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fury 0.9.0 requires aiohttp>=3.8.4, but you have aiohttp 3.8.1 which is incompatible.
gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2023.12.2 which is incompatible.
tensorflowjs 4.